<a href="https://colab.research.google.com/github/caccioppoli/etas_optimization/blob/main/etas_op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd
import pickle as pkl
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
import time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading instrumental catalog

In [72]:
with open("drive/MyDrive/Colab Notebooks/cat_italia.txt","r") as infile, open("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",'w') as outfile:
    for line in infile:
      outfile.write(" ".join(line.split()).replace('|',' '))
      outfile.write("\n")

sdate = np.genfromtxt("drive/MyDrive/Colab Notebooks/cat_italia_conv.txt",dtype="str",delimiter=" ",usecols=(1))
startdate = datetime.strptime('1985-01-02T18:39:30.740000',"%Y-%m-%dT%H:%M:%S.%f")
seconds = []
for time in sdate:
  date = datetime.strptime(time,"%Y-%m-%dT%H:%M:%S.%f")
  temp = (date - startdate).total_seconds()
  seconds.append(temp)








      


[0.0, 15492.35, 22006.25, 166689.5, 192024.08, 205652.9, 222791.16, 228836.06, 239222.37, 292372.64, 294005.91, 331149.83, 341007.8, 357259.78, 366918.55, 392066.07, 404904.32, 437715.23, 441844.91, 465818.85, 481842.41, 530802.33, 536043.39, 536345.05, 539610.92, 549210.35, 562252.9, 580505.06, 584459.17, 619974.03, 643307.57, 665651.4, 680056.66, 693789.37, 694213.99, 712273.03, 784265.79, 787468.72, 839138.58, 860501.96, 863132.58, 920065.35, 920784.14, 950645.78, 978007.86, 1036153.77, 1127659.99, 1170779.77, 1201834.85, 1227388.42, 1228205.6, 1230217.82, 1259217.78, 1278948.58, 1309535.56, 1354270.74, 1355099.71, 1452089.75, 1457727.35, 1476969.93, 1494734.45, 1565273.24, 1579135.78, 1585109.51, 1596925.7, 1617360.97, 1659605.92, 1686237.4, 1740248.28, 1752290.12, 1783190.64, 1783847.32, 1805524.22, 2000800.37, 2004821.47, 2105113.14, 2210782.46, 2488652.08, 2498388.61, 2545340.93, 2550741.09, 2582710.3, 2583235.62, 2586611.85, 2595008.74, 2598174.44, 2640833.06, 2646166.36, 26787